In [1]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from glob import glob
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import random
import albumentations as A
from PIL import Image
import cv2

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend
from tensorflow.keras import layers
from google.colab import auth
auth.authenticate_user()
tf.config.set_soft_device_placement(True)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  print('Device:', tpu.master())
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
  strategy = tf.distribute.get_strategy()
print('Number of replicas', strategy.num_replicas_in_sync)

Autotune = tf.data.experimental.AUTOTUNE

print(tf.__version__)

GCS_PATH = "gs://hoon_bari"

AUTOTUNE = tf.data.experimental.AUTOTUNE

file_path = tf.io.gfile.glob(str(GCS_PATH + '/Dacon_wallpaper/*.tfrec'))
# 파일 갯수 확인
print('TFRecord Files:', len(file_path))

Device: grpc://10.3.238.74:8470


Number of replicas 8
2.12.0
TFRecord Files: 0


In [ ]:
# 랜덤 시드 설정
seed_value = 1337

# Python의 시드 설정
random.seed(seed_value)

# Numpy의 시드 설정
np.random.seed(seed_value)

# TensorFlow의 시드 설정
tf.random.set_seed(seed_value)

In [4]:
!pip uninstall -y pillow
!CC="cc -mavx2" pip install -U --force-reinstall pillow-simd

Found existing installation: Pillow 8.4.0
Uninstalling Pillow-8.4.0:
  Successfully uninstalled Pillow-8.4.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.9/849.9 kB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pillow-simd: filename=Pillow_SIMD-9.0.0.post1-cp310-cp310-linux_x86_64.whl size=1446037 sha256=0e02ec0315f790b5f253963f17b93e35dde57ec4196e19e654e58cfb5c764c0f
  Stored in directory: /root/.cache/pip/wheels/5b/40/80/c3580fdb22f9cc7362b26cac344b501994785f96329a3f6b94
Successfully built pillow-simd


In [ ]:
# 설치 후 런타임 재실행
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
base_dir = '/content/drive/MyDrive/Dacon_wallpaper/'

In [ ]:
train_folder = glob(base_dir + 'train/*')

train_path = []
for folder in train_folder:
    tmp = glob(folder + '/*')
    train_path += tmp

In [ ]:
train_df = pd.DataFrame(train_path, columns=['path'])
train_df['label'] = train_df['path'].apply(lambda x: x.split('/')[-2])
train_df

,path,label
0,/content/drive/MyDrive/Dacon_wallpaper/train/ᄀ...,걸레받이수정
1,/content/drive/MyDrive/Dacon_wallpaper/train/ᄀ...,걸레받이수정
2,/content/drive/MyDrive/Dacon_wallpaper/train/ᄀ...,걸레받이수정
3,/content/drive/MyDrive/Dacon_wallpaper/train/ᄀ...,걸레받이수정
4,/content/drive/MyDrive/Dacon_wallpaper/train/ᄀ...,걸레받이수정
...,...,...
3452,/content/drive/MyDrive/Dacon_wallpaper/train/ᄒ...,훼손
3453,/content/drive/MyDrive/Dacon_wallpaper/train/ᄒ...,훼손
3454,/content/drive/MyDrive/Dacon_wallpaper/train/ᄒ...,훼손
3455,/content/drive/MyDrive/Dacon_wallpaper/train/ᄒ...,훼손


In [ ]:
train_df['label'].value_counts()

훼손                 1405
오염                  595
걸레받이수정        307
꼬임                  210
터짐                  162
곰팡이               145
오타공                142
몰딩수정            130
면불량               99
석고수정              57
들뜸                  54
피스                    51
창틀,문틀수정      27
울음                  22
이음부불량           17
녹오염                14
가구수정               12
틈새과다                5
반점                   3
Name: label, dtype: int64

In [ ]:
import matplotlib.pyplot as plt

label_counts = train_df['label'].value_counts()
plt.bar(range(len(label_counts)), label_counts.values)
plt.xticks(range(len(label_counts)), label_counts.index)
plt.xlabel('Class')
plt.ylabel('Number of Images')
plt.title('Distribution of Train Dataset')
plt.show()

In [ ]:
plt.figure(figsize = (15,12))
for idx, i in enumerate(train_df.label.unique()):
    plt.subplot(4, 7, idx+1)
    df = train_df[train_df['label'] == i].reset_index(drop = True)
    # image_path = df.loc[random.randint(0, len(df))-1, 'path']
    image_path = df.loc[random.randint(0, len(df)-1), 'path']
    img = Image.open(image_path)
    img = img.resize((224,224))
    plt.imshow(img)
    plt.axis('off')
    plt.title(i)
plt.tight_layout()
plt.show()

In [ ]:
# augmentation 데이터셋 생성기
transform = A.Compose([
    A.ShiftScaleRotate(scale_limit=(0, 0.1), p=0.7),
    A.RandomBrightnessContrast(brightness_limit=[-0.3, 0.1], contrast_limit=[-0.3, 0.1], p=1),
    A.HorizontalFlip(p=0.5),
    A.GaussNoise(var_limit=(10, 50), p=0.5),
    A.CoarseDropout(p=0.3, max_holes=15, max_height=15, max_width=15),
    A.OneOf([
        A.CLAHE(p=0.7),
        A.ToGray(p=0.1),
        A.Blur(blur_limit=(5, 10), p=0.2)
    ], p=1)
])

# augmentation을 적용할 클래스 폴더들
class_folders = ['가구수정', '걸레받이수정', '곰팡이', '꼬임', '오염', '녹오염', '들뜸', '면불량', '몰딩수정', '반점', '석고수정', '오타공', '울음', '이음부불량', '창틀,문틀수정', '터짐', '틈새과다', '피스', '훼손']

# augmentation을 적용할 클래스 폴더들을 순회하며 augmentation 적용
for class_folder in tqdm(class_folders):
    # 클래스 폴더 경로
    class_dir = f'/content/drive/MyDrive/Dacon_wallpaper/train/{class_folder}'

    # augmentation된 이미지를 저장할 디렉토리 경로
    aug_dir = f'/content/drive/MyDrive/Dacon_wallpaper/train_augmented/{class_folder}'

    # 폴더 생성
    if not os.path.exists(aug_dir):
        os.makedirs(aug_dir)

    if class_folder == '훼손':
        num = 0
        aug = 1705 - len(os.listdir(class_dir))
        while num < aug: 
            for idx, filename in enumerate(os.listdir(class_dir)):
                # 이미지 파일을 불러와 augmentation 적용
                img_path = os.path.join(class_dir, filename)
                img = np.array(Image.open(img_path))
                augmented = transform(image=img)
                img_augmented = augmented['image']

                # augmentation된 이미지 저장
                Image.fromarray(img_augmented).save(os.path.join(aug_dir, f'augmented_{num+1}.png'))
                num += 1
                if num == aug:
                    break

    elif class_folder == '오염':
        num = 0
        aug = 895 - len(os.listdir(class_dir))
        while num < aug: 
            for idx, filename in enumerate(os.listdir(class_dir)):
                # 이미지 파일을 불러와 augmentation 적용
                img_path = os.path.join(class_dir, filename)
                img = np.array(Image.open(img_path))
                augmented = transform(image=img)
                img_augmented = augmented['image']

                # augmentation된 이미지 저장
                Image.fromarray(img_augmented).save(os.path.join(aug_dir, f'augmented_{num+1}.png'))
                num += 1
                if num == aug:
                    break

    elif class_folder == '걸레받이수정':
        num = 0
        aug = 500 - len(os.listdir(class_dir))
        while num < aug: 
            for idx, filename in enumerate(os.listdir(class_dir)):
                # 이미지 파일을 불러와 augmentation 적용
                img_path = os.path.join(class_dir, filename)
                img = np.array(Image.open(img_path))
                augmented = transform(image=img)
                img_augmented = augmented['image']

                # augmentation된 이미지 저장
                Image.fromarray(img_augmented).save(os.path.join(aug_dir, f'augmented_{num+1}.png'))
                num += 1
                if num == aug:
                    break

        
    elif class_folder == '꼬임':
        num = 0
        aug = 400 - len(os.listdir(class_dir))
        while num < aug: 
            for idx, filename in enumerate(os.listdir(class_dir)):
                # 이미지 파일을 불러와 augmentation 적용
                img_path = os.path.join(class_dir, filename)
                img = np.array(Image.open(img_path))
                augmented = transform(image=img)
                img_augmented = augmented['image']

                # augmentation된 이미지 저장
                Image.fromarray(img_augmented).save(os.path.join(aug_dir, f'augmented_{num+1}.png'))
                num += 1
                if num == aug:
                    break
    
    elif class_folder == '터짐':
        num = 0
        aug = 350 - len(os.listdir(class_dir))
        while num < aug: 
            for idx, filename in enumerate(os.listdir(class_dir)):
                # 이미지 파일을 불러와 augmentation 적용
                img_path = os.path.join(class_dir, filename)
                img = np.array(Image.open(img_path))
                augmented = transform(image=img)
                img_augmented = augmented['image']

                # augmentation된 이미지 저장
                Image.fromarray(img_augmented).save(os.path.join(aug_dir, f'augmented_{num+1}.png'))
                num += 1
                if num == aug:
                    break

    elif class_folder == '곰팡이' or class_folder == '오타공' or class_folder == '몰딩수정' or class_folder == '면불량' or class_folder == '석고수정':
        num = 0
        aug = 300 - len(os.listdir(class_dir))
        while num < aug: 
            for idx, filename in enumerate(os.listdir(class_dir)):
                # 이미지 파일을 불러와 augmentation 적용
                img_path = os.path.join(class_dir, filename)
                img = np.array(Image.open(img_path))
                augmented = transform(image=img)
                img_augmented = augmented['image']

                # augmentation된 이미지 저장
                Image.fromarray(img_augmented).save(os.path.join(aug_dir, f'augmented_{num+1}.png'))
                num += 1
                if num == aug:
                    break


    elif class_folder == '들뜸' or class_folder == '피스' or class_folder == '울음' or class_folder == '창틀,문틀수정':
        num = 0
        aug = 200 - len(os.listdir(class_dir))
        while num < aug: 
            for idx, filename in enumerate(os.listdir(class_dir)):
                # 이미지 파일을 불러와 augmentation 적용
                img_path = os.path.join(class_dir, filename)
                img = np.array(Image.open(img_path))
                augmented = transform(image=img)
                img_augmented = augmented['image']

                # augmentation된 이미지 저장
                Image.fromarray(img_augmented).save(os.path.join(aug_dir, f'augmented_{num+1}.png'))
                num += 1
                if num == aug:
                    break
    
    elif class_folder == '이음부불량' or class_folder == '가구수정' or class_folder == '틈새과다':
        num = 0
        aug = 150 - len(os.listdir(class_dir))
        while num < aug: 
            for idx, filename in enumerate(os.listdir(class_dir)):
                # 이미지 파일을 불러와 augmentation 적용
                img_path = os.path.join(class_dir, filename)
                img = np.array(Image.open(img_path))
                augmented = transform(image=img)
                img_augmented = augmented['image']

                # augmentation된 이미지 저장
                Image.fromarray(img_augmented).save(os.path.join(aug_dir, f'augmented_{num+1}.png'))
                num += 1
                if num == aug:
                    break
    
    elif class_folder == '녹오염':
        num = 0
        aug = 100 - len(os.listdir(class_dir))
        while num < aug: 
            for idx, filename in enumerate(os.listdir(class_dir)):
                # 이미지 파일을 불러와 augmentation 적용
                img_path = os.path.join(class_dir, filename)
                img = np.array(Image.open(img_path))
                augmented = transform(image=img)
                img_augmented = augmented['image']

                # augmentation된 이미지 저장
                Image.fromarray(img_augmented).save(os.path.join(aug_dir, f'augmented_{num+1}.png'))
                num += 1
                if num == aug:
                    break

    elif class_folder == '반점':
        num = 0
        aug = 100 - len(os.listdir(class_dir))
        while num < aug: 
            for idx, filename in enumerate(os.listdir(class_dir)):
                # 이미지 파일을 불러와 augmentation 적용
                img_path = os.path.join(class_dir, filename)
                img = np.array(Image.open(img_path))
                augmented = transform(image=img)
                img_augmented = augmented['image']

                # augmentation된 이미지 저장
                Image.fromarray(img_augmented).save(os.path.join(aug_dir, f'augmented_{num+1}.png'))
                num += 1
                if num == aug:
                    break

100%|██████████| 19/19 [09:46<00:00, 30.86s/it]


In [ ]:
# augmentation data 개수 확인
aug_folder = glob(base_dir + 'train_augmented/*')

aug_path = []
for folder in aug_folder:
    tmp = glob(folder + '/*')
    aug_path += tmp

aug_df = pd.DataFrame(aug_path, columns=['path'])
aug_df['label'] = aug_df['path'].apply(lambda x: x.split('/')[-2])
aug_df['label'].value_counts()

훼손                 300
오염                 300
석고수정            243
면불량             201
걸레받이수정       193
꼬임                 190
터짐                 188
울음                178
창틀,문틀수정    173
몰딩수정           170
오타공               158
곰팡이              155
피스                  149
들뜸                146
틈새과다             145
가구수정             138
이음부불량         133
반점                 97
녹오염               86
Name: label, dtype: int64

In [ ]:
# 총 파일 개수 확인
df = pd.concat([train_df, aug_df], ignore_index=True)
df

,path,label
0,/content/drive/MyDrive/Dacon_wallpaper/train/ᄀ...,걸레받이수정
1,/content/drive/MyDrive/Dacon_wallpaper/train/ᄀ...,걸레받이수정
2,/content/drive/MyDrive/Dacon_wallpaper/train/ᄀ...,걸레받이수정
3,/content/drive/MyDrive/Dacon_wallpaper/train/ᄀ...,걸레받이수정
4,/content/drive/MyDrive/Dacon_wallpaper/train/ᄀ...,걸레받이수정
...,...,...
6795,/content/drive/MyDrive/Dacon_wallpaper/train_a...,훼손
6796,/content/drive/MyDrive/Dacon_wallpaper/train_a...,훼손
6797,/content/drive/MyDrive/Dacon_wallpaper/train_a...,훼손
6798,/content/drive/MyDrive/Dacon_wallpaper/train_a...,훼손


In [3]:
from sklearn.preprocessing import LabelEncoder

# 학습 데이터셋 경로
train_img_dir = '/content/drive/MyDrive/Dacon_wallpaper/train/'

# augmentation 데이터셋 경로
aug_train_img_dir = '/content/drive/MyDrive/Dacon_wallpaper/train_augmented/'

# 이미지 크기 설정
img_width, img_height = 384, 384

# 데이터셋 경로와 레이블 추출
dataset_paths = []
labels = []

for label in os.listdir(train_img_dir):
    label_path = os.path.join(train_img_dir, label)
    img_files = os.listdir(label_path)

    if label == '훼손':
        label_path = os.path.join(train_img_dir, label)
        img_files = os.listdir(label_path)
        img_files = random.sample(img_files, min(800, len(img_files)))
        for img_file in img_files:
            img_path = os.path.join(label_path, img_file)
            dataset_paths.append(img_path)
            labels.append(label)

    elif label == '오염':
        label_path = os.path.join(train_img_dir, label)
        img_files = os.listdir(label_path)
        img_files = random.sample(img_files, min(400, len(img_files)))
        for img_file in img_files:
            img_path = os.path.join(label_path, img_file)
            dataset_paths.append(img_path)
            labels.append(label)
        
    else:
        label_path = os.path.join(train_img_dir, label)
        for img_file in os.listdir(label_path):
            img_path = os.path.join(label_path, img_file)
            dataset_paths.append(img_path)
            labels.append(label)


for label in os.listdir(aug_train_img_dir):
    label_path = os.path.join(aug_train_img_dir, label)
    for img_file in os.listdir(label_path):
        img_path = os.path.join(label_path, img_file)
        dataset_paths.append(img_path)
        labels.append(label)

In [4]:
len(dataset_paths) # 6000장 확인

6000

In [5]:
# 데이터 셋 불러오기
def load_dataset(dataset_paths, img_width, img_height):
    x_data = []
    for path in tqdm(dataset_paths):
        img = Image.open(path)
        img = img.resize((img_width, img_height))
        img = np.asarray(img)
        # 이미지 전처리
        img = tf.keras.applications.efficientnet_v2.preprocess_input(img)
        x_data.append(img)

    x_data = np.array(x_data)
    return x_data

x_data = load_dataset(dataset_paths, img_width, img_height)
y_data = np.array(labels)

# 레이블 인코딩
le = LabelEncoder()
y_encoded = le.fit_transform(y_data)

100%|██████████| 6000/6000 [03:11<00:00, 31.41it/s]


In [6]:
# hybrid Model(efficientNet, swin, 출처 : https://github.com/innat/HybridModel-GradCAM/blob/main/notebooks/%5BColab%5D_HENetSwinT.ipynb)

patch_size      = (2,2)   # 2-by-2 sized patches
dropout_rate    = 0.5     # Dropout rate
num_heads       = 8       # Attention heads
embed_dim       = 64      # Embedding dimension
num_mlp         = 128     # MLP layer size
qkv_bias        = True    # Convert embedded patches to query, key, and values
window_size     = 2       # Size of attention window
shift_size      = 1       # Size of shifting window
image_dimension = 24      # Initial image size / Input size of the transformer model 

num_patch_x = image_dimension // patch_size[0]
num_patch_y = image_dimension // patch_size[1]

In [7]:
def window_partition(x, window_size):
    _, height, width, channels = x.shape
    patch_num_y = height // window_size
    patch_num_x = width // window_size
    x = tf.reshape(
        x, shape=(-1, patch_num_y, window_size, patch_num_x, window_size, channels)
    )
    x = tf.transpose(x, (0, 1, 3, 2, 4, 5))
    windows = tf.reshape(x, shape=(-1, window_size, window_size, channels))
    return windows


def window_reverse(windows, window_size, height, width, channels):
    patch_num_y = height // window_size
    patch_num_x = width // window_size
    x = tf.reshape(
        windows,
        shape=(-1, patch_num_y, patch_num_x, window_size, window_size, channels),
    )
    x = tf.transpose(x, perm=(0, 1, 3, 2, 4, 5))
    x = tf.reshape(x, shape=(-1, height, width, channels))
    return x


class DropPath(layers.Layer):
    def __init__(self, drop_prob=None, **kwargs):
        super(DropPath, self).__init__(**kwargs)
        self.drop_prob = drop_prob

    def call(self, inputs, training=None):
        if self.drop_prob == 0.0 or not training:
            return inputs
        else:
            batch_size = tf.shape(inputs)[0]
            keep_prob = 1 - self.drop_prob
            path_mask_shape = (batch_size,) + (1,) * (len(tf.shape(inputs)) - 1)
            path_mask = tf.floor(
                backend.random_bernoulli(path_mask_shape, p=keep_prob)
            )
            outputs = (
                tf.math.divide(tf.cast(inputs, dtype=tf.float32), keep_prob) * path_mask
            )
            return outputs

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "drop_prob": self.drop_prob,
            }
        )
        return config

In [8]:
class PatchExtract(layers.Layer):
    def __init__(self, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.patch_size_x = patch_size[0]
        self.patch_size_y = patch_size[0]

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=(1, self.patch_size_x, self.patch_size_y, 1),
            strides=(1, self.patch_size_x, self.patch_size_y, 1),
            rates=(1, 1, 1, 1),
            padding="VALID",
        )
        patch_dim = patches.shape[-1]
        patch_num = patches.shape[1]
        return tf.reshape(patches, (batch_size, patch_num * patch_num, patch_dim))

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "patch_size_y": self.patch_size_y,
                "patch_size_x": self.patch_size_x,
            }
        )
        return config


class PatchEmbedding(layers.Layer):
    def __init__(self, num_patch, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.num_patch = num_patch
        self.proj = layers.Dense(embed_dim)
        self.pos_embed = layers.Embedding(input_dim=num_patch, output_dim=embed_dim)

    def call(self, patch):
        pos = tf.range(start=0, limit=self.num_patch, delta=1)
        return self.proj(patch) + self.pos_embed(pos)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "num_patch": self.num_patch,
            }
        )
        return config


class PatchMerging(layers.Layer):
    def __init__(self, num_patch, embed_dim):
        super().__init__()
        self.num_patch = num_patch
        self.embed_dim = embed_dim
        self.linear_trans = layers.Dense(2 * embed_dim, use_bias=False)

    def call(self, x):
        height, width = self.num_patch
        _, _, C = x.get_shape().as_list()
        x = tf.reshape(x, shape=(-1, height, width, C))
        feat_maps = x

        x0 = x[:, 0::2, 0::2, :]
        x1 = x[:, 1::2, 0::2, :]
        x2 = x[:, 0::2, 1::2, :]
        x3 = x[:, 1::2, 1::2, :]
        x = tf.concat((x0, x1, x2, x3), axis=-1)
        x = tf.reshape(x, shape=(-1, (height // 2) * (width // 2), 4 * C))
        return self.linear_trans(x), feat_maps

    def get_config(self):
        config = super().get_config()
        config.update({"num_patch": self.num_patch, "embed_dim": self.embed_dim})
        return config

In [9]:
class WindowAttention(layers.Layer):
    def __init__(
        self,
        dim,
        window_size,
        num_heads,
        qkv_bias=True,
        dropout_rate=0.0,
        return_attention_scores=False,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.dim = dim
        self.window_size = window_size
        self.num_heads = num_heads
        self.scale = (dim // num_heads) ** -0.5
        self.return_attention_scores = return_attention_scores
        self.qkv = layers.Dense(dim * 3, use_bias=qkv_bias)
        self.dropout = layers.Dropout(dropout_rate)
        self.proj = layers.Dense(dim)

    def build(self, input_shape):
        self.relative_position_bias_table = self.add_weight(
            shape=(
                (2 * self.window_size[0] - 1) * (2 * self.window_size[1] - 1),
                self.num_heads,
            ),
            initializer="zeros",
            trainable=True,
            name="relative_position_bias_table",
        )

        self.relative_position_index = self.get_relative_position_index(
            self.window_size[0], self.window_size[1]
        )
        super().build(input_shape)

    def get_relative_position_index(self, window_height, window_width):
        x_x, y_y = tf.meshgrid(range(window_height), range(window_width))
        coords = tf.stack([y_y, x_x], axis=0)
        coords_flatten = tf.reshape(coords, [2, -1])

        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]
        relative_coords = tf.transpose(relative_coords, perm=[1, 2, 0])

        x_x = (relative_coords[:, :, 0] + window_height - 1) * (2 * window_width - 1)
        y_y = relative_coords[:, :, 1] + window_width - 1
        relative_coords = tf.stack([x_x, y_y], axis=-1)

        return tf.reduce_sum(relative_coords, axis=-1)

    def call(self, x, mask=None):
        _, size, channels = x.shape
        head_dim = channels // self.num_heads
        x_qkv = self.qkv(x)
        x_qkv = tf.reshape(x_qkv, shape=(-1, size, 3, self.num_heads, head_dim))
        x_qkv = tf.transpose(x_qkv, perm=(2, 0, 3, 1, 4))
        q, k, v = x_qkv[0], x_qkv[1], x_qkv[2]
        q = q * self.scale
        k = tf.transpose(k, perm=(0, 1, 3, 2))
        attn = q @ k

        relative_position_bias = tf.gather(
            self.relative_position_bias_table,
            self.relative_position_index,
            axis=0,
        )
        relative_position_bias = tf.transpose(relative_position_bias, [2, 0, 1])
        attn = attn + tf.expand_dims(relative_position_bias, axis=0)

        if mask is not None:
            nW = mask.get_shape()[0]
            mask_float = tf.cast(
                tf.expand_dims(tf.expand_dims(mask, axis=1), axis=0), tf.float32
            )
            attn = (
                tf.reshape(attn, shape=(-1, nW, self.num_heads, size, size))
                + mask_float
            )
            attn = tf.reshape(attn, shape=(-1, self.num_heads, size, size))
            attn = tf.nn.softmax(attn, axis=-1)
        else:
            attn = tf.nn.softmax(attn, axis=-1)
        attn = self.dropout(attn)

        x_qkv = attn @ v
        x_qkv = tf.transpose(x_qkv, perm=(0, 2, 1, 3))
        x_qkv = tf.reshape(x_qkv, shape=(-1, size, channels))
        x_qkv = self.proj(x_qkv)
        x_qkv = self.dropout(x_qkv)

        if self.return_attention_scores:
            return x_qkv, attn
        else:
            return x_qkv

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "dim": self.dim,
                "window_size": self.window_size,
                "num_heads": self.num_heads,
                "scale": self.scale,
            }
        )
        return config

In [10]:
import jax
from jax import jit
from jax import random
from jax import numpy as jnp
from jax.experimental import jax2tf

class SwinTransformer(layers.Layer):
    def __init__(
        self, 
        dim,
        num_patch,
        num_heads,
        window_size=7,
        shift_size=0,
        num_mlp=1024,
        qkv_bias=True,
        dropout_rate=0.0,
        **kwargs,
    ):
        super(SwinTransformer, self).__init__(**kwargs)

        self.dim = dim 
        self.num_patch = num_patch  
        self.num_heads = num_heads 
        self.window_size = window_size  
        self.shift_size = shift_size  
        self.num_mlp = num_mlp  

        self.norm1 = layers.LayerNormalization(epsilon=1e-5)
        self.attn = WindowAttention(
            dim,
            window_size=(self.window_size, self.window_size),
            num_heads=num_heads,
            qkv_bias=qkv_bias,
            dropout_rate=dropout_rate,
        )
        self.drop_path = (
            DropPath(dropout_rate) if dropout_rate > 0.0 else tf.identity
        )
        self.norm2 = layers.LayerNormalization(epsilon=1e-5)

        self.mlp = keras.Sequential(
            [
                layers.Dense(num_mlp),
                layers.Activation(keras.activations.gelu),
                layers.Dropout(dropout_rate),
                layers.Dense(dim),
                layers.Dropout(dropout_rate),
            ]
        )

        if min(self.num_patch) < self.window_size:
            self.shift_size = 0
            self.window_size = min(self.num_patch)

    def build(self, input_shape):
        if self.shift_size == 0:
            self.attn_mask = None
        else:
            height, width = self.num_patch
            h_slices = (
                slice(0, -self.window_size),
                slice(-self.window_size, -self.shift_size),
                slice(-self.shift_size, None),
            )
            w_slices = (
                slice(0, -self.window_size),
                slice(-self.window_size, -self.shift_size),
                slice(-self.shift_size, None),
            )
            mask_array = jnp.zeros((1, height, width, 1))
            count = 0
            for h in h_slices:
                for w in w_slices:
                    mask_array[:, h, w, :] = count
                    count += 1
            mask_array = tf.convert_to_tensor(mask_array)

            # mask array to windows
            mask_windows = window_partition(mask_array, self.window_size)
            mask_windows = tf.reshape(
                mask_windows, shape=[-1, self.window_size * self.window_size]
            )
            attn_mask = tf.expand_dims(mask_windows, axis=1) - tf.expand_dims(
                mask_windows, axis=2
            )
            attn_mask = tf.where(attn_mask != 0, -100.0, attn_mask)
            attn_mask = tf.where(attn_mask == 0, 0.0, attn_mask)
            self.attn_mask = tf.Variable(initial_value=attn_mask, trainable=False)

    def call(self, x):
        height, width = self.num_patch
        _, num_patches_before, channels = x.shape
        x_skip = x
        x = self.norm1(x)
        x = tf.reshape(x, shape=(-1, height, width, channels))
        if self.shift_size > 0:
            shifted_x = tf.roll(
                x, shift=[-self.shift_size, -self.shift_size], axis=[1, 2]
            )
        else:
            shifted_x = x

        x_windows = window_partition(shifted_x, self.window_size)
        x_windows = tf.reshape(
            x_windows, shape=(-1, self.window_size * self.window_size, channels)
        )
        attn_windows = self.attn(x_windows, mask=self.attn_mask)

        attn_windows = tf.reshape(
            attn_windows, shape=(-1, self.window_size, self.window_size, channels)
        )
        shifted_x = window_reverse(
            attn_windows, self.window_size, height, width, channels
        )
        if self.shift_size > 0:
            x = tf.roll(
                shifted_x, shift=[self.shift_size, self.shift_size], axis=[1, 2]
            )
        else:
            x = shifted_x

        x = tf.reshape(x, shape=(-1, height * width, channels))
        x = self.drop_path(x)
        x = tf.cast(x_skip, dtype=tf.float32) + tf.cast(x, dtype=tf.float32)
        x_skip = x
        x = self.norm2(x)
        x = self.mlp(x)
        x = self.drop_path(x)
        x = tf.cast(x_skip, dtype=tf.float32) + tf.cast(x, dtype=tf.float32)
        return x

In [11]:
# creating a custom train_step to override the train_step method of sub_classing model
def train_step_sam(self, data, rho=0.05):
    """
    Overrides the train_step method of Model
    
    Args:
        data : Data on which model is to be trained
        rho  : Hyperparameter Rho indicating the size of neighborhood
    """
    
    sample_weight = None
    x, y = data

    # Opening Gradient Tape scope to record operations during 1st forward pass
    with tf.GradientTape() as tape:
        y_pred = self(x, training=True)
        # Calculating loss to calculate gradients
        loss = self.compiled_loss(y, y_pred, sample_weight=sample_weight, regularization_losses=self.losses)

    
    trainable_vars = self.trainable_variables
    # Calculating gradients with respect trainable variable
    gradients = tape.gradient(loss, trainable_vars)

    """
    This is the first step which involves calculating the point w_adv with highest loss and virtually moving to that point so that we can get gradient at that point. 
    """
    eps_w_ls = [] # list to store the updates done to trainable variables in first step
    
    #computing the norm of gradients which is required for computing eps_w
    grad_norm = tf.linalg.global_norm(gradients)
    
    # Iterating over trainable_vars
    for i in range(len(trainable_vars)):
        # we will calculate eps_w to find w_adv point having highest loss in rho neighborhood
        eps_w = tf.math.multiply(gradients[i], rho / grad_norm )
        # temporarily moving to w_adv point
        trainable_vars[i].assign_add(eps_w)
        # storing updates done in eps_w_ls list 
        eps_w_ls.append(eps_w)

    # Opening Gradient Tape scope to record operations during 2nd forward pass
    with tf.GradientTape() as tape:
        y_pred = self(x, training=True) 
        # Calculating loss to calculate gradient at w_adv point
        loss = self.compiled_loss(y, y_pred, sample_weight=sample_weight, regularization_losses=self.losses)
        
    trainable_vars = self.trainable_variables
    #computing gradient at w_adv which is our objective in this first step
    gradients = tape.gradient(loss, trainable_vars)

    """
    This is the second step in SAM where we will do actual update at the initial point from the gradient calculated at adversial point w_adv 
    """
    
    for i in range(len(trainable_vars)):
        # Going back to orignal parameters
        trainable_vars[i].assign_sub(eps_w_ls[i])
    
    # Updating parameters with gradients computed at w_adv
    self.optimizer.apply_gradients(zip(gradients, trainable_vars))

    # Updating the metrics.
    self.compiled_metrics.update_state(y, y_pred, sample_weight=sample_weight)

    # returns a dictionary mapping metric names (including the loss) to their current value.
    return {m.name: m.result() for m in self.metrics}

In [41]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization, LayerNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2M
from sklearn.metrics import f1_score
from tensorflow.keras.losses import categorical_crossentropy

with strategy.scope():
    base = EfficientNetV2M(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

    class HybridModel(keras.Model):
        def __init__(self, model_name, **kwargs):
            super().__init__(name=model_name, **kwargs)

            # base model with compatible output which will be an input of transformer model
            self.multi_output_cnn = keras.Model(
                [base.inputs],
                [base.get_layer("block6a_expand_activation").output, base.output],
                name="efficientnet",
            )

            # base model's (cnn model) head
            self.conv_head = keras.Sequential(
                [
                    layers.GlobalAveragePooling2D(),
                    layers.AlphaDropout(0.5),
                    layers.LayerNormalization()
                ],
                name="conv_head",
            )

            # stuff of swin transformers
            self.patch_extract = PatchExtract(patch_size)
            self.patch_embedds = PatchEmbedding(num_patch_x * num_patch_y, embed_dim)
            self.patch_merging = PatchMerging(
                (num_patch_x, num_patch_y), embed_dim=embed_dim
            )

            # swin blocks containers
            self.swin_sequences = keras.Sequential(name="swin_blocks")
            for i in range(shift_size):
                self.swin_sequences.add(
                    SwinTransformer(
                        dim=embed_dim,
                        num_patch=(num_patch_x, num_patch_y),
                        num_heads=num_heads,
                        window_size=window_size,
                        shift_size=i,
                        num_mlp=num_mlp,
                        qkv_bias=qkv_bias,
                        dropout_rate=dropout_rate,
                    )
                )

            # swin block's head
            self.swin_head = keras.Sequential(
                [
                    layers.GlobalAveragePooling1D(),
                    layers.AlphaDropout(0.5),
                    layers.LayerNormalization(),
                ],
                name="swin_head",
            )

            # classifier
            self.classifier = layers.Dense(
                19, activation='softmax'
            )

            # build the graph
            self.build_graph()

        def forward_cnn(self, inputs):
            # CNN model.
            return self.multi_output_cnn(inputs)

        def forward_transformer(self, inputs):
            # Transformer model.
            x = self.patch_extract(inputs)
            x = self.patch_embedds(x)
            x = self.swin_sequences(tf.cast(x, dtype=tf.float32))
            x, swin_gcam_top = self.patch_merging(x)
            return x, swin_gcam_top

        def call(self, inputs, training=None, **kwargs):
            cnn_mid_layer, cnn_gcam_top = self.forward_cnn(inputs)
            transformer_output, transformer_gcam_top = self.forward_transformer(
                cnn_mid_layer
            )

            transformer_output = self.swin_head(transformer_output)
            cnn_output = self.conv_head(cnn_gcam_top)
            logits = self.classifier(tf.concat([transformer_output, cnn_output], axis=-1))

            return logits

        def build_graph(self):
            x = keras.Input(shape=(img_width, img_height, 3))
            return keras.Model(inputs=[x], outputs=self.call(x))
        
        # overriding the train_step method  with our custom train_step_sam created in earlier cell
        def train_step(self, data):
            return train_step_sam(self, data, rho=0.05) # using rho as 0.05 you can tune this hyperparameter

    # focal loss 정의
    def focal_loss(gamma=2.0, alpha=0.25):
        def loss(y_true, y_pred):
            ce_loss = categorical_crossentropy(y_true, y_pred)
            pt = tf.math.exp(-ce_loss)
            focal_loss = alpha * tf.math.pow(1. - pt, gamma) * ce_loss
            return focal_loss
        return loss

    model = HybridModel("efficientnet")
    model.build(input_shape=(None, img_height, img_width, 3))

    model.compile(
        optimizer = tf.keras.optimizers.AdamW(learning_rate=0.0001),
        loss = focal_loss(),
        metrics=['accuracy'])

In [42]:
model.summary()

Model: "efficientnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnet (Functional)   [(None, 24, 24, 1056),    53150388  
                              (None, 12, 12, 1280)]              
                                                                 
 conv_head (Sequential)      (None, 1280)              2560      
                                                                 
 patch_extract_5 (PatchExtra  (None, 144, 4224)        0         
 ct)                                                             
                                                                 
 patch_embedding_5 (PatchEmb  (None, 144, 64)          279616    
 edding)                                                         
                                                                 
 patch_merging_5 (PatchMergi  ((None, 36, 128),        32768     
 ng)                          (None, 12, 12, 64))     

In [43]:
from sklearn.model_selection import StratifiedKFold

# 콜백들
earlystopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',
    min_delta = 0.001,
    patience=8,
    restore_best_weights=True,
    verbose=0)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
checkpoint_dir = f'gs://hoon_bari/Dacon_wallpaper/Training model'
best_f1_score = 0
best_weights = None

for fold, (train_index, val_index) in enumerate(kfold.split(x_data, y_encoded)):
    
    # 학습 데이터와 검증 데이터를 나누기
    x_train, y_train = x_data[train_index], y_encoded[train_index]
    x_val, y_val = x_data[val_index], y_encoded[val_index]

    model.fit(x_train, tf.keras.utils.to_categorical(y_train), 
                batch_size=32, 
                epochs=30,
                validation_data=(x_val, tf.keras.utils.to_categorical(y_val)),
                callbacks=[earlystopping, reduce_lr])

    # 모델 예측
    y_pred = model.predict(x_val)
    y_pred_label = np.argmax(y_pred, axis=1)

    # 모델 평가
    score_f1 = f1_score(y_val, y_pred_label, average='weighted')
    print(f'Fold {fold+1} - f1_score:', score_f1)

    # 가장 좋은 f1_score를 보인 모델 저장
    if score_f1 > best_f1_score:
        best_f1_score = score_f1
        best_weights = model.get_weights()

# 모델 checkpoint 저장
checkpoint_name = f'{checkpoint_dir}/'
model.set_weights(best_weights)
model.save_weights(checkpoint_name)

Epoch 1/30
150/150 [==============================] - 469s 642ms/step - loss: 0.6650 - accuracy: 0.1921 - val_loss: 0.7479 - val_accuracy: 0.6217 - lr: 1.0000e-04
Epoch 2/30
150/150 [==============================] - 54s 357ms/step - loss: 0.4146 - accuracy: 0.4031 - val_loss: 0.6097 - val_accuracy: 0.7425 - lr: 1.0000e-04
Epoch 3/30
150/150 [==============================] - 54s 359ms/step - loss: 0.2756 - accuracy: 0.5648 - val_loss: 0.5164 - val_accuracy: 0.8167 - lr: 1.0000e-04
Epoch 4/30
150/150 [==============================] - 53s 354ms/step - loss: 0.1949 - accuracy: 0.6690 - val_loss: 0.4893 - val_accuracy: 0.8467 - lr: 1.0000e-04
Epoch 5/30
150/150 [==============================] - 53s 354ms/step - loss: 0.1306 - accuracy: 0.7654 - val_loss: 0.4462 - val_accuracy: 0.8650 - lr: 1.0000e-04
Epoch 6/30
150/150 [==============================] - 53s 351ms/step - loss: 0.1027 - accuracy: 0.8115 - val_loss: 0.4053 - val_accuracy: 0.8917 - lr: 1.0000e-04
Epoch 7/30
150/150 [=======

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

x_train, x_val, y_train, y_val = train_test_split(x_data, y_encoded, test_size=0.3, shuffle=True, stratify=y_encoded, random_state=42)

# train set 예측
train_pred = model.predict(x_train)
train_pred_classes = np.argmax(train_pred, axis=1)

# train set 평가
print("Train Set 평가 결과")
print(classification_report(y_train, train_pred_classes))

# validation set 예측
val_pred = model.predict(x_val)
val_pred_classes = np.argmax(val_pred, axis=1)

# validation set 평가
print("Validation Set 평가 결과")
print(classification_report(y_val, val_pred_classes))

132/132 [==============================] - 8s 55ms/step
Train Set 평가 결과
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       105
           1       1.00      1.00      1.00       350
           2       0.98      1.00      0.99       210
           3       1.00      0.99      0.99       280
           4       1.00      1.00      1.00        70
           5       0.98      0.99      0.99       140
           6       0.99      0.97      0.98       210
           7       0.98      1.00      0.99       210
           8       0.99      1.00      0.99        70
           9       1.00      0.99      1.00       210
          10       0.97      0.97      0.97       490
          11       1.00      0.99      0.99       210
          12       0.95      1.00      0.97       140
          13       0.96      1.00      0.98       105
          14       0.99      1.00      1.00       140
          15       0.99      0.98      0.99       245
         

In [45]:
df = pd.DataFrame(index=range(0, 0), columns=['id', 'label'])

In [46]:
test_csv = pd.read_csv('/content/drive/MyDrive/Dacon_wallpaper/test.csv')

In [47]:
test_csv

,id,img_path
0,TEST_000,./test/000.png
1,TEST_001,./test/001.png
2,TEST_002,./test/002.png
3,TEST_003,./test/003.png
4,TEST_004,./test/004.png
...,...,...
787,TEST_787,./test/787.png
788,TEST_788,./test/788.png
789,TEST_789,./test/789.png
790,TEST_790,./test/790.png


In [48]:
%cd /content/drive/MyDrive/Dacon_wallpaper

/content/drive/MyDrive/Dacon_wallpaper


In [49]:
# 클래스 라벨
class_names = ['가구수정', '걸레받이수정', '곰팡이', '꼬임', '녹오염', '들뜸', '면불량', '몰딩수정', '반점', '석고수정', '오염', '오타공', '울음', '이음부불량', '창틀,문틀수정', '터짐', '틈새과다', '피스', '훼손']

# 이미지 파일 불러오기
test_paths = test_csv['img_path'].values
test_images = load_dataset(test_paths, img_width, img_height)

# 모델 예측
predictions = model.predict(test_images)

# 예측 결과 데이터프레임에 저장
predicted_labels = [class_names[i] for i in np.argmax(predictions, axis=1)]
test_csv['label'] = predicted_labels
df = test_csv[['id', 'label']]

100%|██████████| 792/792 [00:11<00:00, 66.98it/s]


25/25 [==============================] - 14s 562ms/step


In [54]:
df

,id,label
0,TEST_000,훼손
1,TEST_001,오염
2,TEST_002,훼손
3,TEST_003,몰딩수정
4,TEST_004,오염
...,...,...
787,TEST_787,오염
788,TEST_788,오염
789,TEST_789,오염
790,TEST_790,오염


In [55]:
df['label'].nunique()

19

In [56]:
df['label'].value_counts()

훼손         321
오염         229
꼬임          29
면불량         28
터짐          23
울음          22
몰딩수정        21
곰팡이         18
오타공         18
이음부불량       16
피스          13
걸레받이수정      11
창틀,문틀수정     10
가구수정         9
석고수정         8
들뜸           6
반점           4
녹오염          4
틈새과다         2
Name: label, dtype: int64

In [53]:
# DataFrame을 csv 파일로 저장
df.to_csv('submission.csv', index=False)